<a href="https://colab.research.google.com/github/Brycexxx/fastai/blob/master/lesson3_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lesson3

### 1. [fastai](https://docs.fast.ai/) 中一些 api 的参数解释

- [get_transforms](https://docs.fast.ai/vision.transform.html#Data-augmentation-details) 

  - do_flip：翻转，默认为 true。将以 0.5 的概率随即水平翻转图片，例如在猫狗识别中，猫或狗朝左还是朝右都是可以的，这样的翻转可以增加数据的广泛性，但是不能上下翻转，猫或狗是不会倒着站立的。
  - flip_vert：垂直翻转，默认为 false。若设置为 true，则 `do_flip` 必须为 true，图片可被垂直翻转，或者旋转 90&deg; ，为 false 则只有水平方向的翻转
  - max_warp：弯曲，扭曲。这个不太好理解，举个例子，在猫狗识别中，照片拍摄的角度实际上是很不同的，比如仰拍一只狗和俯拍在一只狗，其实狗在照片中呈现的外形是有些不一致的，所以就可应用这个参数做数据增强；但是在行星数据集中，由于都是卫星在高空拍摄，所以就不会有猫狗那种情况发生，此时需要关闭这个参数

- [create_cnn](https://docs.fast.ai/vision.learner.html#create_cnn) 的 `metrics` 

  - `metrics` 可以有多种指标，放入一个列表中，他们对于模型的结果不会有任何改变，只是在训练的过程中方便我们知道模型的变化。

  - 当模型是多标签分类模型时，一个图片输入可能输出多个标签，此时一般的 `accuracy` 方法就不管用了，由源码可以知道此方法使用 `argmax` 只会输出一个最大后验概率对应的索引，不能满足输出多个索引的要求，此时就要用到 `accuracy_thresh` ，这个方法可以输入特定的后验概率阈值，达到阈值即代表输入图片具有这个输出特征，即标签，这样就可以输出多个标签。

    - 一般做法：比如取阈值为 0.2

      ```python
      def acc_02(input, target):
          return accuracy_thresh(input, target, thresh=0.2)
      ```

    - 实际上 python3 提供了更便捷的做法，`partial` 函数可以传入一个函数和需要固定的函数参数，返回的是已经固定了某个参数的原函数

      ```python
      acc_02 = partial(accuracy_thresh, thresh=0.2)
      ```

### 2. 对于已经部署的 Web app

可以将用户上传图片分类错误的记录下来，并让用户告知这属于哪一个类别。然后利用错误分类的输入创建新的数据，对部署的模型进行微调。这时候可能需要稍微大一点的学习率或者更多的 `epoch` 使得微调更加有效

### 3. 对于给定的图片数据集可以先使用更小的尺寸

使用更小的尺寸可以更加快速的进行实验，在得到一个还不错的结果时，再使用更大的尺寸；此时我们也不必从头开始，可以使用**迁移学习**在小尺寸图片训练出来的模型上进行微调，提高训练的效率；同时，在某种程度上来说，迁移学习还减少了过拟合的风险。比如最开始使用的尺寸是 128 x 128，在进行了一些列的实验之后开始进行迁移学习微调，图片尺寸放大到 256 x 256，实际上相当于原输入图片的 4 倍，这对于卷积神经网络来说可以说是完全不同的数据集，所以说在一定程度上减少了过拟合的风险。使用更小的尺寸先进行预训练是一个很棒的技巧，目前还并不广为人所知。

### 4. 关于学习率的设定

- 一个属于的解释 `ground truth` ：标准答案

- [fastai](https://docs.fast.ai/) 中的 `fit_one_cycle` 方法中的学习率从一个较小的值开始，然后变大再变小，如下如所示：

  <img src="https://raw.githubusercontent.com/Brycexxx/Images/master/20190206193704.jpg"/>

- 学习率的衰退是一个很早就开始采取的策略，也很好理解，因为模型参数逐渐收敛到最优，我们需要一个更小的学习步长靠近最优解，以免跨步太大而错过；然而在开始时逐渐提高学习率的做法却是最近才提出来的，这个做法主要帮助了模型在开始探索整个损失函数颠簸的表面，效果如下：

  <img src="https://raw.githubusercontent.com/Brycexxx/Images/master/20190206195705.gif"/>

- 实际上 `fit_one_cycle` 方法传入的不是一个确定的学习率，而是最大的学习率。当训练完一个模型时，如果模型在训练集和验证集上的损失都是先微微增大然后越来越小，这就意味着你找到了一个不错的最大学习率，如下图：

  <img src="https://raw.githubusercontent.com/Brycexxx/Images/master/20190206200722.jpg"/>

  如果在训练的过程中，损失一直保持下降，那么在解冻之后，你最好稍微提高学习率，因为这样有利于跳出局部最优解，得到上面的损失曲线。

### 5. 混合精度训练

如果显卡内存被用完，则可以考虑使用混合精度训练。混合精度训练的意思是训练模型的过程中大部分计算都可以用半精度浮点数完成也就是 16 bits 而不是 32 bits，这减少了 GPU 内存的使用，意味着你可以使用更大的 batch 或者更快的训练模型。

### 6. 预训练模型的使用与正规化统计数据的选择问题

- 要明确的是：如果我们使用了预训练模型，则正规化对应的统计数据则必须选择预训练模型对应的统计数据，而不能使用自身的均值和方差等统计信息
- 为什么：举个例子，假如我们在做一个识别不同品种绿色青蛙的模型，如果分别使用三个通道的均值和方差进行正规化，那么最后得到的是每个通道的数据均值为 0，方差为 1，这意味着这些图片看起来不再像绿色青蛙，更像灰色青蛙，所以使用自身的统计数据进行正规化，则数据中独特的特征将会消失，所以要使用和预训练模型一致的统计数据。（那 imagenet 在训练的时候使用的正规化又是怎样的呢？？？不使用预训练模型，又该怎么正规化呢？）
- 对于不使用预训练模型的训练，就使用自身数据的均值和方差来进行正规化，即使用了预训练模型就要使用预训练模型对应的均值和方差进行正规化，未使用的就是用自身的均值和方差，至于为什么，还没想得特别明白！！！

### 7. 广义逼近理论

如果将足够多的线性函数和非线性函数堆在一起则可以逼近任意的函数

### 8. 如何处理 2 通道的图片和 4 通道的图片

- 对于 2 通道的图片，我们可以手动的创建第三个通道，全部算作 0 或者可以使用另外，另外两个通道的平均数
- 对于 4 通道的图片，我们需要改变模型而不是图片数据，比如对于普通的三通道图片，我们的初始权重张量的 shape 可能为 5 x 5 x 3 x 6，5 x 5 代表单个过滤器的尺寸，3 代表输入图片的通道数，6 代表使用的过滤器数量，所以当我们遇到 4 通道图片时，我们可以相应地把 3 变为 4，即权重张量为  5 x 5 x 4 x 6

